In [2]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [3]:
# 读取20220831的所有数据
all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_35274/1764276053.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)


In [ ]:
# 将raw_cvjd_20220831_spark.csv的数据精简，只取有用的列
# new_columns = ['cv_id', 'jd_id', 'title', 'category_name', 'recruit_number', 'min_month_salary', 'max_month_salary', 'pay_months', 'tags', 'description', 'requirement', 'currentPosition', ]

In [10]:
all_data

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,updatedBy,updatedTime,uploaderId,version,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,67,2021-12-16 17:25:39,67,1,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,JD20210918160449291911,会员管理经理,2021-10-31 23:59:59.0,130101,用户运营,2c9207157b9b9199017bc895b2f23a39,上海晨光文具股份有限公司,1,...,2c9207157bcd68cf017bf309f6cb003c,2021-09-22 21:00:12,2c9207157bcd68cf017bf309f6cb003c,1,7.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2014-06"", ""id"":...","['{""companyName"": ""北京美餐好客科技有限公司"", ""companyNatu...","['{""description"": ""项目描述: 母婴类人群，包含童装、童鞋、奶粉。\\n运...",20220831
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,JD20210923104433723714,产品市场经理,2021-10-31 23:59:59.0,110101,产品经理,2c9207157bf94d99017c108765c804aa,凯斯纽荷兰（中国）管理有限公司,1,...,121,2021-10-12 09:51:56,121,1,14.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2012-09"", ""id"":...","['{""companyName"": ""Continental Tires (China) L...",NaN,20220831
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,2c9207157bcd68cf017becb0f28b0025,2021-11-23 10:16:45,2c9207157bcd68cf017becb0f28b0025,1,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,28,2021-12-15 13:47:16,28,1,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,JD20220822112125679712,品牌负责人,NaN,130402,运营总监,8a69c4df82b081d30182c37f39d00627,浙江温家味食品科技有限公司,1,...,8a69f61e82926af00182c9b341b36129,2022-08-23 17:15:26,8a69f61e82926af00182c9b341b36129,1,11.0,NaN,"['{""endDate"": ""2012-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-06"", ""companyName"": ""江苏领迅食...","['{""name"": null}']",20220831
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,JD20220818193847553940,大区销售总监,NaN,140402,销售总监,8a69c47b82b07fdc0182b0b5e27a6357,上海普适导航科技股份有限公司,5,...,8a69c4b9820f193401821984f2517399,2022-08-25 13:50:02,8a69c4b9820f193401821984f2517399,1,14.0,NaN,"['{""endDate"": ""2008-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""安徽庆宇光...","['{""name"": null}']",20220831
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,2c92071581ec1aab0181f0475082756f,2022-08-22 10:59:17,2c92071581ec1aab0181f0475082756f,1,5.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""泰猎人力(...","['{""description"": ""项目描述: 客服云智库是河南电信客服人员内部使用的一套...",20220831
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,JD20220629161617600853,Legal Senior Manager,NaN,150506,法务经理,2c9207157f347800017f8d372b237e20,曼恩商用车辆贸易（中国）有限公司,1,...,2c9207157f40e97a017f71b1c7c11d59,2022-08-31 18:36:40,2c9207157bf92566017c164de933001c,2,16.0,NaN,"

In [12]:
df = pd.read_csv('../data_20220831/sample_20220831_spark.csv')
df

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_35274/394408529.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data_20220831/sample_20220831_spark.csv')


,Unnamed: 0,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,...,industry_cut,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,C972256920524165184,JD20210910134407574550,drop,1,1,市场策划总监,2021-11-30 23:59:59.0,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['舞蹈', '学']","[0.2290763258934021, -0.14894524216651917, 0.0...","[-0.033116504549980164, -0.058041930198669434,...","['[', '""', 'Ai', '""', ',', '""', 'Ar', '""', ','...","[0.37405160069465637, 0.08108291774988174, -0....","[0.18812726438045502, -0.014001657254993916, -...",20220831
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,C972256920532553817,JD20210916103438584949,link,-1,1,解决方案总监/经理,2021-10-31 23:59:59.0,...,"['通信', '技术', '提供商']","[0.18720732629299164, -0.24131253361701965, -0...","[0.768592119216919, -0.16999222338199615, -0.4...",['计算机技术'],"[0.06550661474466324, -0.30920350551605225, -0...","[0.00835432019084692, -0.6533162593841553, -0....","['[', '""', 'it', '规划', '咨询', '""', ',', '""', '云...","[0.32306215167045593, 0.09080911427736282, -0....","[0.5299833416938782, -0.1909421980381012, -0.2...",20220831
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,C972256920545136700,JD20220216104801688663,drop,1,1,医学博士投资人,NaN,...,"['基金', '/', '证券', '/', '期货', '/', '投资']","[0.022260336205363274, -0.03374471887946129, -...","[0.30257120728492737, -0.31254786252975464, -0...","['生物医学', '工程']","[0.01792692393064499, -0.1300506740808487, -0....","[0.005150213837623596, 0.15490269660949707, -0...","['[', '""', 'VC', '""', ',', '""', 'PPT', '""', ',...","[0.3084435760974884, 0.07542910426855087, -0.2...","[0.09389838576316833, 0.04999205097556114, -0....",20220831
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,C972256920557719589,JD20210908152800055103,reject,2,2,高级用户运营专家,2021-12-31 23:59:59.0,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['市场营销'],"[0.6256594657897949, -0.07856443524360657, 0.0...","[0.7786060571670532, -0.8168095946311951, -0.6...","['[', '""', 'Vi', '""', ',', '""', 'Pr', '""', ','...","[0.3983086943626404, 0.11806842684745789, -0.2...","[0.29188045859336853, -0.04354493319988251, -0...",20220831
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,C972256920557719638,JD20211011173942648108,accept,3,3,咨询总监,2021-10-31 23:59:59.0,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['金融学'],"[0.11331989616155624, -0.1490677297115326, -0....","[-0.3050949275493622, -0.4833742380142212, -2....","['[', '""', 'PPT', '""', ',', '""', 'Word', '""', ...","[0.4078637957572937, 0.07882367819547653, -0.2...","[0.1595805436372757, -0.0552152618765831, -0.3...",20220831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83296,83296,8a69d7c28219945301823a12e0266463,8a69c4df82b081d30182c9ec1ecd1c67,C1001558237016887296,JD20220823165719130900,link,-1,1,声学工程师,NaN,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['韩国', '永进', '大学']","[-0.016853807494044304, -0.16189433634281158, ...","[0.4639318585395813, 0.4096287488937378, 0.103...","['CAD', ',', 'Proe', ',', 'Ansys', ',', '优化', ...","[0.29724088311195374, -0.06454341858625412, -0...","[-0.07170484960079193, 0.2823019027709961, -0....",20220831
83297,83297,8a69d7c282199453018246ebf6b60643,8a69f67f82198d89018234adfcd91510,C1002462334712680448,JD20220725172633862905,post,-1,2,销售代表,NaN,...,"['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['河南', '科技', '大学']","[0.43664801120758057, -0.4357180

In [2]:
# 读入20220821所有数据
#all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_67195/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [6]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [7]:
all_data[['work_age', 'workYear']]

,work_age,workYear
0,5-10年,13.0
1,3-5年,7.0
2,10年以上,14.0
3,3-5年,3.0
4,3-5年,5.0
...,...,...
83296,5年以上,11.0
83297,5-10年,14.0
83298,3-5年,5.0
83299,3-5年,16.0


## 理清需要用的特征


In [8]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 'pay_months', 'currentSalary', 'desiredSalary']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_words', 'equal_job']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [9]:
all_data[['cv_id', 'jd_id']+raw_columns]

,cv_id,jd_id,age,recruit_number,workYear,min_annual_salary,max_annual_salary,pay_months,currentSalary,desiredSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,80.0,150.0,NaN,NaN,NaN
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,28.0,35.0,NaN,NaN,NaN
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,25.0,35.0,NaN,NaN,NaN
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,60.0,500.0,NaN,NaN,NaN
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,NaN,NaN,12.0,保密,25 - 30k · 15薪
...,...,...,...,...,...,...,...,...,...,...
83296,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,NaN,NaN,12.0,"月薪2.98左右,15薪",合理涨幅
83297,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,NaN,NaN,12.0,12万（10000元/月*,10K~13K
83298,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,NaN,NaN,14.0,18k,20-25k
83299,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,NaN,NaN,13.0,90万,90万


## tfidf_pca

In [1]:
import pandas as pd
# 读入20220821所有数据
all_data = pd.read_csv('../data_20220831/raw_cvjd_20220831_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_28916/358587841.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [2]:
sample = all_data

# 需要将这个向量变成一个列表
def tfidf_pca_merge_fun(series):
    return list(series)

tfidf_pca_file = ['description_tfidf_pca.csv', 'position_tfidf_pca.csv', 'requirement_tfidf_pca.csv', 'skills_tfidf_pca.csv', 'title_category_tags_tfidf_pca.csv']
for csv_file in tfidf_pca_file:
    tfidf_pca = pd.read_csv('../data_20220831/'+csv_file).drop(['Unnamed: 0'], axis=1)
    print(csv_file)
    #print(tfidf_pca)
    col_name = csv_file.strip('.csv') # 把skills的s去掉了
    tfidf_pca[col_name] = tfidf_pca.apply(tfidf_pca_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_pca[[col_name]]], axis=1)

tfidf_pca_col_names = [csv_file.strip('.csv') for csv_file in tfidf_pca_file]
sample[['cv_id', 'jd_id']+tfidf_pca_col_names]

description_tfidf_pca.csv
position_tfidf_pca.csv
requirement_tfidf_pca.csv
skills_tfidf_pca.csv
title_category_tags_tfidf_pca.csv


,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062337206435, 0.048274327757322, 0.060...","[-0.0393278200938532, -0.1156517718522584, -0....","[-0.0884550099224587, -0.1255198001313325, -0....","[0.3299248933717911, 0.1647421276349334, 0.188...","[-0.0428002835031876, -0.1584211550818529, -0...."
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193630086019, 0.0553739702927152, -0.1...","[0.1054180773841507, -0.0263972179315595, -0.0...","[-0.0803609391957365, -0.1239125520897878, -0....","[0.2365059748734542, 0.1136815904753885, -0.23...","[-0.2368437481203857, 0.0858239404855124, -0.2..."
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212087673029, -0.1819505382209111, -0....","[-0.0368150744632823, -0.0868620764628543, -0....","[-0.0644952418491031, -0.1617720033381562, -0....","[-0.0915394265067953, -0.1409956060493181, -0....","[-0.1908741363785154, 0.1941298571078743, 0.34..."
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188847607561, 0.0989579068236976, -0.0...","[0.0490419846480027, -0.0412213763728095, -0.0...","[-0.1351532034332717, 0.1740612193389041, 0.04...","[0.1429876430780753, 0.0219409230262712, -0.14...","[-0.3720984260133064, 0.1912182010657805, -0.3..."
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442478713477, -0.0330071782675384, 0.06...","[-0.0271020069629156, -0.1132012426106846, -0....","[-0.0138174189038416, 0.1003273435064088, -0.0...","[-0.0992961449703319, -0.1495964264638832, -0....","[0.0056217550187971, -0.0571648001001755, 0.00..."
...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.061454010695859, -0.0867970871691909, -0.0...","[-0.0745961612097089, -0.0529118806357184, -0....","[-0.0400398546050934, -0.0616278628416963, 0.0...","[-0.0986273363552968, -0.1820013678668413, -0....","[0.0344861195736259, -0.0468052249094291, 0.05..."
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[0.0151640431728487, -0.0802995343655144, 0.00..."
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[-0.0053327004788714, -0.0845578341662708, 0.0..."
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.4074295016834148, -0.0564815622372111, 0.38...","[-0.3849339451379412, 0.4677351578603559, 0.34...","[0.5339787831331781, 0.0783524285243356, 0.129...","[-0.2529650781905327, 0.1589143771515023, 0.08...","[0.547197647548344, 0.4049099774125437, -0.158..."


## work_duration

In [3]:
work_duration_mean_df = pd.read_csv('../data_20220831/work_duration_mean.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, work_duration_mean_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062337206435, 0.048274327757322, 0.060...","[-0.0393278200938532, -0.1156517718522584, -0....","[-0.0884550099224587, -0.1255198001313325, -0....","[0.3299248933717911, 0.1647421276349334, 0.188...","[-0.0428002835031876, -0.1584211550818529, -0....",1209.500000
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193630086019, 0.0553739702927152, -0.1...","[0.1054180773841507, -0.0263972179315595, -0.0...","[-0.0803609391957365, -0.1239125520897878, -0....","[0.2365059748734542, 0.1136815904753885, -0.23...","[-0.2368437481203857, 0.0858239404855124, -0.2...",506.000000
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212087673029, -0.1819505382209111, -0....","[-0.0368150744632823, -0.0868620764628543, -0....","[-0.0644952418491031, -0.1617720033381562, -0....","[-0.0915394265067953, -0.1409956060493181, -0....","[-0.1908741363785154, 0.1941298571078743, 0.34...",811.500000
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188847607561, 0.0989579068236976, -0.0...","[0.0490419846480027, -0.0412213763728095, -0.0...","[-0.1351532034332717, 0.1740612193389041, 0.04...","[0.1429876430780753, 0.0219409230262712, -0.14...","[-0.3720984260133064, 0.1912182010657805, -0.3...",76.500000
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442478713477, -0.0330071782675384, 0.06...","[-0.0271020069629156, -0.1132012426106846, -0....","[-0.0138174189038416, 0.1003273435064088, -0.0...","[-0.0992961449703319, -0.1495964264638832, -0....","[0.0056217550187971, -0.0571648001001755, 0.00...",1034.000000
...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.061454010695859, -0.0867970871691909, -0.0...","[-0.0745961612097089, -0.0529118806357184, -0....","[-0.0400398546050934, -0.0616278628416963, 0.0...","[-0.0986273363552968, -0.1820013678668413, -0....","[0.0344861195736259, -0.0468052249094291, 0.05...",91.000000
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[0.0151640431728487, -0.0802995343655144, 0.00...",1234.857143
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[-0.0053327004788714, -0.0845578341662708, 0.0...",1234.857143
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.4074295016834148, -0.0564815622372111, 0.38...","[-0.3849339451379412, 0.4677351578603559, 0.34...","[0.5339787831331781, 0.0783524285243356, 0.129...","[-0.2529650781905327, 0.1589143771515023, 0.08...","[0.547197647548344, 0.4049099774125437, -0.158...",598.666667


## equal_words

In [4]:
equal_job_df = pd.read_csv('../data_20220831/equal_job.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, equal_job_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062337206435, 0.048274327757322, 0.060...","[-0.0393278200938532, -0.1156517718522584, -0....","[-0.0884550099224587, -0.1255198001313325, -0....","[0.3299248933717911, 0.1647421276349334, 0.188...","[-0.0428002835031876, -0.1584211550818529, -0....",1209.500000,3
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193630086019, 0.0553739702927152, -0.1...","[0.1054180773841507, -0.0263972179315595, -0.0...","[-0.0803609391957365, -0.1239125520897878, -0....","[0.2365059748734542, 0.1136815904753885, -0.23...","[-0.2368437481203857, 0.0858239404855124, -0.2...",506.000000,4
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212087673029, -0.1819505382209111, -0....","[-0.0368150744632823, -0.0868620764628543, -0....","[-0.0644952418491031, -0.1617720033381562, -0....","[-0.0915394265067953, -0.1409956060493181, -0....","[-0.1908741363785154, 0.1941298571078743, 0.34...",811.500000,0
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188847607561, 0.0989579068236976, -0.0...","[0.0490419846480027, -0.0412213763728095, -0.0...","[-0.1351532034332717, 0.1740612193389041, 0.04...","[0.1429876430780753, 0.0219409230262712, -0.14...","[-0.3720984260133064, 0.1912182010657805, -0.3...",76.500000,3
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442478713477, -0.0330071782675384, 0.06...","[-0.0271020069629156, -0.1132012426106846, -0....","[-0.0138174189038416, 0.1003273435064088, -0.0...","[-0.0992961449703319, -0.1495964264638832, -0....","[0.0056217550187971, -0.0571648001001755, 0.00...",1034.000000,1
...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.061454010695859, -0.0867970871691909, -0.0...","[-0.0745961612097089, -0.0529118806357184, -0....","[-0.0400398546050934, -0.0616278628416963, 0.0...","[-0.0986273363552968, -0.1820013678668413, -0....","[0.0344861195736259, -0.0468052249094291, 0.05...",91.000000,2
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[0.0151640431728487, -0.0802995343655144, 0.00...",1234.857143,3
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[-0.0053327004788714, -0.0845578341662708, 0.0...",1234.857143,4
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.4074295016834148, -0.0564815622372111, 0.38...","[-0.3849339451379412, 0.4677351578603559, 0.34...","[0.5339787831331781, 0.0783524285243356, 0.129...","[-0.2529650781905327, 0.1589143771515023, 0.08...","[0.547197647548344, 0.4049099774125437, -0.158...",598.666667,5


## salary

In [5]:
salary_df = pd.read_csv('../data_20220831/salary.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, salary_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job'] + ['parsed_desiredSalary', 'parsed_currentSalary']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062337206435, 0.048274327757322, 0.060...","[-0.0393278200938532, -0.1156517718522584, -0....","[-0.0884550099224587, -0.1255198001313325, -0....","[0.3299248933717911, 0.1647421276349334, 0.188...","[-0.0428002835031876, -0.1584211550818529, -0....",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193630086019, 0.0553739702927152, -0.1...","[0.1054180773841507, -0.0263972179315595, -0.0...","[-0.0803609391957365, -0.1239125520897878, -0....","[0.2365059748734542, 0.1136815904753885, -0.23...","[-0.2368437481203857, 0.0858239404855124, -0.2...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212087673029, -0.1819505382209111, -0....","[-0.0368150744632823, -0.0868620764628543, -0....","[-0.0644952418491031, -0.1617720033381562, -0....","[-0.0915394265067953, -0.1409956060493181, -0....","[-0.1908741363785154, 0.1941298571078743, 0.34...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188847607561, 0.0989579068236976, -0.0...","[0.0490419846480027, -0.0412213763728095, -0.0...","[-0.1351532034332717, 0.1740612193389041, 0.04...","[0.1429876430780753, 0.0219409230262712, -0.14...","[-0.3720984260133064, 0.1912182010657805, -0.3...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442478713477, -0.0330071782675384, 0.06...","[-0.0271020069629156, -0.1132012426106846, -0....","[-0.0138174189038416, 0.1003273435064088, -0.0...","[-0.0992961449703319, -0.1495964264638832, -0....","[0.0056217550187971, -0.0571648001001755, 0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.061454010695859, -0.0867970871691909, -0.0...","[-0.0745961612097089, -0.0529118806357184, -0....","[-0.0400398546050934, -0.0616278628416963, 0.0...","[-0.0986273363552968, -0.1820013678668413, -0....","[0.0344861195736259, -0.0468052249094291, 0.05...",91.000000,2,-1,-1
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[0.0151640431728487, -0.0802995343655144, 0.00...",1234.857143,3,-1,-1
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626293402285, 0.2009878730596399, 0.10...","[-0.0248770952780892, -0.0818758200756602, -0....","[-0.1178366575446526, 0.1256630731223134, 0.14...","[0.0156514071533009, 0.0238285547103318, -0.06...","[-0.0053327004788714, -0.0845578341662708, 0.0...",1234.857143,4,-1,-1
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.4074295016834148, -0.0564815622372111, 0.38...","[-0.3849339451379412, 0.4677351578603559, 0.34...","[0.5339787831331781, 0.0783524285243356, 0.129...","[-0.2529650781905327, 0.1589143771515023, 0.08...","[0.547197647548344, 0.4049099774125437, -0.158...",598.666667,5,336,300


## 数据集划分，剔除复推集